In [ ]:
#| default_exp callbacks

# Callbacks
> Callback used in handlers

In [ ]:
#| export
import fastcore.all as fc
from operator import attrgetter
from cftime import date2num

from marisco.configs import get_cfgs

In [ ]:
#| export
class Callback(): order = 0

In [ ]:
#| export
def run_cbs(cbs, obj=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        if cb.__doc__: obj.logs.append(cb.__doc__)
        cb(obj)

In [ ]:
#| export
class Transformer():
    def __init__(self, dfs, cbs=None): 
        fc.store_attr()
        self.logs = []
        
    def callback(self):
        run_cbs(self.cbs, self)
        
    def __call__(self):
        self.callback()
        return self.dfs

## Generic

In [ ]:
#| export
class EncodeTimeCB(Callback):
    "Encode time as `int` representing seconds since xxx"

    def __call__(self, tfm):
        def format_time(x): return date2num(x, units=get_cfgs('units')['time'])
        for k in tfm.dfs.keys():
            tfm.dfs[k]['time'] = tfm.dfs[k]['time'].apply(format_time)

In [ ]:
#| export
class SanitizeLonLatCB(Callback):
    "Drop row when both longitude & latitude equal 0"

    def __call__(self, tfm):
        tfm.dfs = {grp: (df[(df.lon != 0) & (df.lat != 0)])
                   for grp, df in tfm.dfs.items()}